<a href="https://colab.research.google.com/github/aarsanjani/meansquares/blob/master/version2/feature_engineering/CA_FeatureRanking_Gini_SupersetData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Goal:
Using Random Forest to estimate the gini index and feature importance for all features from CA **Superset** dataset.
Details: 

>* **Gini-Coefficient:** Used the code from our machine learning. Uses CA combined data and Maryland data for gini coefficient score
>* **Random Forest (Gini Score):** Random classifier needs a target value. Hence, *CA-CDC labeled data* is used to get feature importance score. **Target: Growth_label_Code**
>* **Extra Tree Classifier (Gini Score):** ET classifier needs a target value. Hence *CA-CDC labeled data* is used to get feature importance score. **Target: Growth_label_Code**



## Mount Google drive

In [1]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import seaborn as sns
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler as ss
import itertools
from scipy import linalg
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import export_graphviz
import numpy as np
import matplotlib.pyplot as plt

# 1.Load data - Combined CA Data

## 1.1Preprocessed Data

In [3]:
CA_superset_data = pd.read_csv('/content/drive/Shared drives/CMPE 295- Master Project/projectdata-2021/CA_Superset_dataset_Feb12.csv',low_memory=False)
CA_superset_data.head()

,Date,Province_State,FIPS,New cases,County Name,mask_rule_severity,m50,m50_index,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,PopDensity,State_Name,State,CTFIPS,CTNAME,STFIPS,Social.distancing.index,X..staying.home,Trips.person,X..out.of.county.trips,X..out.of.state.trips,Miles.person,Work.trips.person,Non.work.trips.person,New.COVID.cases,Population,X..change.in.consumption,Transit.mode.share,X..people.older.than.60,Median.income,X..African.Americans,X..Hispanic.Americans,X..Male,Employment.density,X..hot.spots.1000.people,Hospital.beds.1000.people,ICUs.1000.people,X..contact.tracing.workers.1000.people,New.cases.1000.people,Active.cases.1000.people,X.days..decreasing.COVID.cases,X..hospital.bed.utilization,Testing.capacity,Tests.done.1000.people,X..ICU.utilization,Ventilator.shortage,Imported.COVID.cases,COVID.exposure.1000.people,X.days..decreasing.ILI.cases,Unemployment.claims.1000.people,Unemployment.rate,X..working.from.home,Cumulative.inflation.rate,COVID.death.rate
0,2020-03-01,California,6001,0,Alameda County,0.0,3.684,58,6.0,9.0,15.0,1.0,4.0,0.0,860.757075,California,CA,6001,Alameda County,6,37,26,2.93,25.4,0.4,36.8,0.25,2.68,0,1666753,-5.1,15.04,19,92574,10.5,22.5,49.11,1155,131,2.05,0.22,NaN,0.0000,0.0,37,54.0,0.0,0.00,0.0,0,0,0.05,35,1.1,5.3,5.7,0.6,0.00
1,2020-03-02,California,6001,0,Alameda County,0.0,6.182,98,4.0,7.0,7.0,-1.0,3.0,0.0,860.757075,California,CA,6001,Alameda County,6,17,20,3.38,28.2,0.4,38.1,0.63,2.74,0,1666753,-2.9,15.04,19,92574,10.5,22.5,49.11,1155,131,2.05,0.22,NaN,0.0000,0.0,38,54.0,0.0,0.00,0.0,0,0,0.05,35,1.1,5.3,5.7,0.6,0.00
2,2020-03-03,California,6001,1,Alameda County,0.0,6.271,100,5.0,12.0,41.0,0.0,1.0,-1.0,860.757075,California,CA,6001,Alameda County,6,16,18,3.61,27.1,0.3,32.7,0.67,2.94,1,1666753,4.1,15.04,19,92574,10.5,22.5,49.11,1155,131,2.05,0.22,NaN,0.0002,0.0,0,54.0,0.0,0.00,0.0,0,1,0.05,42,1.1,5.3,5.7,0.6,0.00
3,2020-03-04,California,6001,0,Alameda County,0.0,6.209,99,1.0,6.0,23.0,-3.0,1.0,0.0,860.757075,California,CA,6001,Alameda County,6,14,18,3.55,27.9,0.3,36.9,0.68,2.87,0,1666753,1.7,15.04,19,92574,10.5,22.5,49.11,1155,131,2.05,0.22,NaN,0.0002,0.0,0,54.0,10.3,0.01,0.0,0,1,0.06,42,1.1,5.3,5.7,0.6,0.04
4,2020-03-05,California,6001,0,Alameda County,0.0,6.654,106,1.0,6.0,12.0,-5.0,-1.0,1.0,860.757075,California,CA,6001,Alameda County,6,15,19,3.54,28.1,0.3,36.8,0.66,2.88,0,1666753,2.0,15.04,19,92574,10.5,22.5,49.11,1155,131,2.05,0.22,NaN,0.0002,0.0,0,54.0,10.3,0.01,0.0,0,1,0.06,42,1.1,5.3,5.7,0.6,0.04


In [4]:
CA_superset_data.shape

(15391, 58)

In [5]:
CA_superset_data.dtypes

Date                                                   object
Province_State                                         object
FIPS                                                    int64
New cases                                               int64
County Name                                            object
mask_rule_severity                                    float64
m50                                                   float64
m50_index                                               int64
retail_and_recreation_percent_change_from_baseline    float64
grocery_and_pharmacy_percent_change_from_baseline     float64
parks_percent_change_from_baseline                    float64
transit_stations_percent_change_from_baseline         float64
workplaces_percent_change_from_baseline               float64
residential_percent_change_from_baseline              float64
PopDensity                                            float64
State_Name                                             object
State   

## 1.2Labeled  Data

In [6]:
CA_labeled_data = pd.read_csv('/content/drive/Shared drives/CMPE 295- Master Project/projectdata-2021/CA_SuperData_Labeled_CDCGuideline_Feb13.csv',low_memory=False)
CA_labeled_data.head()

,Date,Province_State,FIPS,New cases,County Name,mask_rule_severity,m50,m50_index,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,PopDensity,State_Name,State,CTFIPS,CTNAME,STFIPS,Social.distancing.index,X..staying.home,Trips.person,X..out.of.county.trips,X..out.of.state.trips,Miles.person,Work.trips.person,Non.work.trips.person,New.COVID.cases,Population,X..change.in.consumption,Transit.mode.share,X..people.older.than.60,Median.income,X..African.Americans,X..Hispanic.Americans,X..Male,Employment.density,X..hot.spots.1000.people,Hospital.beds.1000.people,ICUs.1000.people,X..contact.tracing.workers.1000.people,New.cases.1000.people,Active.cases.1000.people,X.days..decreasing.COVID.cases,X..hospital.bed.utilization,Testing.capacity,Tests.done.1000.people,X..ICU.utilization,Ventilator.shortage,Imported.COVID.cases,COVID.exposure.1000.people,X.days..decreasing.ILI.cases,Unemployment.claims.1000.people,Unemployment.rate,X..working.from.home,Cumulative.inflation.rate,COVID.death.rate,New cases/100k,positivity_rate,rolling_avg_new_cases/100k,growth_label
0,2020-03-01,California,6001,0,Alameda County,0.0,3.684,58,6.0,9.0,15.0,1.0,4.0,0.0,860.757075,California,CA,6001,Alameda County,6,37,26,2.93,25.4,0.4,36.8,0.25,2.68,0,1666753,-5.1,15.04,19,92574,10.5,22.5,49.11,1155,131,2.05,0.22,0.0,0.0000,0.0,37,54.0,0.0,0.00,0.0,0,0,0.05,35,1.1,5.3,5.7,0.6,0.00,0.000000,0.000,0.0,minimal
1,2020-03-02,California,6001,0,Alameda County,0.0,6.182,98,4.0,7.0,7.0,-1.0,3.0,0.0,860.757075,California,CA,6001,Alameda County,6,17,20,3.38,28.2,0.4,38.1,0.63,2.74,0,1666753,-2.9,15.04,19,92574,10.5,22.5,49.11,1155,131,2.05,0.22,0.0,0.0000,0.0,38,54.0,0.0,0.00,0.0,0,0,0.05,35,1.1,5.3,5.7,0.6,0.00,0.000000,0.000,0.0,minimal
2,2020-03-03,California,6001,1,Alameda County,0.0,6.271,100,5.0,12.0,41.0,0.0,1.0,-1.0,860.757075,California,CA,6001,Alameda County,6,16,18,3.61,27.1,0.3,32.7,0.67,2.94,1,1666753,4.1,15.04,19,92574,10.5,22.5,49.11,1155,131,2.05,0.22,0.0,0.0002,0.0,0,54.0,0.0,0.00,0.0,0,1,0.05,42,1.1,5.3,5.7,0.6,0.00,0.059997,0.000,0.0,minimal
3,2020-03-04,California,6001,0,Alameda County,0.0,6.209,99,1.0,6.0,23.0,-3.0,1.0,0.0,860.757075,California,CA,6001,Alameda County,6,14,18,3.55,27.9,0.3,36.9,0.68,2.87,0,1666753,1.7,15.04,19,92574,10.5,22.5,49.11,1155,131,2.05,0.22,0.0,0.0002,0.0,0,54.0,10.3,0.01,0.0,0,1,0.06,42,1.1,5.3,5.7,0.6,0.04,0.000000,0.002,0.0,minimal
4,2020-03-05,California,6001,0,Alameda County,0.0,6.654,106,1.0,6.0,12.0,-5.0,-1.0,1.0,860.757075,California,CA,6001,Alameda County,6,15,19,3.54,28.1,0.3,36.8,0.66,2.88,0,1666753,2.0,15.04,19,92574,10.5,22.5,49.11,1155,131,2.05,0.22,0.0,0.0002,0.0,0,54.0,10.3,0.01,0.0,0,1,0.06,42,1.1,5.3,5.7,0.6,0.04,0.000000,0.002,0.0,minimal


In [7]:
CA_labeled_data.dtypes

Date                           object
Province_State                 object
FIPS                            int64
New cases                       int64
County Name                    object
                               ...   
COVID.death.rate              float64
New cases/100k                float64
positivity_rate               float64
rolling_avg_new_cases/100k    float64
growth_label                   object
Length: 62, dtype: object

In [8]:
CA_labeled_data.shape

(15391, 62)

## Preprocessing data 

* drop na
* convert categorical code

In [9]:
#CA_superset_data = CA_superset_data.dropna()

In [10]:
CA_superset_data.dtypes

Date                                                   object
Province_State                                         object
FIPS                                                    int64
New cases                                               int64
County Name                                            object
mask_rule_severity                                    float64
m50                                                   float64
m50_index                                               int64
retail_and_recreation_percent_change_from_baseline    float64
grocery_and_pharmacy_percent_change_from_baseline     float64
parks_percent_change_from_baseline                    float64
transit_stations_percent_change_from_baseline         float64
workplaces_percent_change_from_baseline               float64
residential_percent_change_from_baseline              float64
PopDensity                                            float64
State_Name                                             object
State   

In [11]:
CA_superset_data.shape

(15391, 58)

## Group the data

In [12]:
CA_superset_data.columns

Index(['Date', 'Province_State', 'FIPS', 'New cases', 'County Name',
       'mask_rule_severity', 'm50', 'm50_index',
       'retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline',
       'parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline',
       'workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline', 'PopDensity', 'State_Name',
       'State', 'CTFIPS', 'CTNAME', 'STFIPS', 'Social.distancing.index',
       'X..staying.home', 'Trips.person', 'X..out.of.county.trips',
       'X..out.of.state.trips', 'Miles.person', 'Work.trips.person',
       'Non.work.trips.person', 'New.COVID.cases', 'Population',
       'X..change.in.consumption', 'Transit.mode.share',
       'X..people.older.than.60', 'Median.income', 'X..African.Americans',
       'X..Hispanic.Americans', 'X..Male', 'Employment.density',
       'X..hot.spots.1000.people', 'Hospital.beds.1000.peop

In [13]:
x_col_demographics = ['PopDensity','X..people.older.than.60',
                      'Median.income', 'X..African.Americans',
                      'X..Hispanic.Americans', 'X..Male','New cases','New.cases.1000.people']

In [14]:
x_col_hospitalization_testing = ['Hospital.beds.1000.people',
       'ICUs.1000.people', 
       'X..hospital.bed.utilization',
       'Testing.capacity', 'Tests.done.1000.people', 'X..ICU.utilization',
       'Ventilator.shortage','New cases','New.cases.1000.people'] #'X..contact.tracing.workers.1000.people',

In [15]:
x_col_mobility = ['retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline','parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline','workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline',
       'X..staying.home', 'Trips.person', 'X..out.of.county.trips',
       'X..out.of.state.trips', 'Miles.person', 'Work.trips.person',
       'Non.work.trips.person','X..working.from.home','New cases','New.cases.1000.people']

In [16]:
x_col_socialdistancing = ['m50', 'm50_index','Social.distancing.index','New cases','New.cases.1000.people']

In [17]:
x_col_maskrule = ['mask_rule_severity','New cases','New.cases.1000.people']

#Demographic Group

## Feature Importance: Gini-Coefficient

In [ ]:
CA_superset_data.columns

Index(['Date', 'Province_State', 'FIPS', 'New cases', 'County Name',
       'mask_rule_severity', 'm50', 'm50_index',
       'retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline',
       'parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline',
       'workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline', 'PopDensity', 'State_Name',
       'State', 'CTFIPS', 'CTNAME', 'STFIPS', 'Social.distancing.index',
       'X..staying.home', 'Trips.person', 'X..out.of.county.trips',
       'X..out.of.state.trips', 'Miles.person', 'Work.trips.person',
       'Non.work.trips.person', 'New.COVID.cases', 'Population',
       'X..change.in.consumption', 'Transit.mode.share',
       'X..people.older.than.60', 'Median.income', 'X..African.Americans',
       'X..Hispanic.Americans', 'X..Male', 'Employment.density',
       'X..hot.spots.1000.people', 'Hospital.beds.1000.peop

In [ ]:
def gini(list_of_values):
    sorted_list = sorted(list_of_values)
    #print (sorted_list)
    height, area = 0, 0
    for value in sorted_list:
        
        height += value
        area += height - value / 2.
    
    fair_area = height * len(list_of_values) / 2.
    print(fair_area)
    return (fair_area - area) / fair_area

In [ ]:
x_col_demographics = ['PopDensity','X..people.older.than.60',
                      'Median.income', 'X..African.Americans',
                      'X..Hispanic.Americans', 'X..Male','New cases','New.cases.1000.people']

In [ ]:
feature_cols= x_col_demographics

gini_values = []
for col in feature_cols:
  #print(col)
  dataset = CA_superset_data[col].values.tolist()
  #print(dataset)
  value = gini(dataset)
  print('cols ',col,' - gini coeff is :',value)
  gini_values.append(value)

35575489453.94788
cols  PopDensity  - gini coeff is : 0.8156637311902494
2688977001.0
cols  X..people.older.than.60  - gini coeff is : 0.14673423679461214
7791742870376.5
cols  Median.income  - gini coeff is : 0.15433249671121319
374657726.1500031
cols  X..African.Americans  - gini coeff is : 0.46161607629775014
3797987049.249782
cols  X..Hispanic.Americans  - gini coeff is : 0.3070973510639532
5971969723.955034
cols  X..Male  - gini coeff is : 0.017576291485370876
14339402229.5
cols  New cases  - gini coeff is : 0.8685321834670556
16852079.173250053
cols  New.cases.1000.people  - gini coeff is : 0.680904683056212


## Feature Importance: Random Forest classifier(Gini Score)

In [ ]:
CA_labeled_data.columns

Index(['Date', 'Province_State', 'FIPS', 'New cases', 'County Name',
       'mask_rule_severity', 'm50', 'm50_index',
       'retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline',
       'parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline',
       'workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline', 'PopDensity', 'State_Name',
       'State', 'CTFIPS', 'CTNAME', 'STFIPS', 'Social.distancing.index',
       'X..staying.home', 'Trips.person', 'X..out.of.county.trips',
       'X..out.of.state.trips', 'Miles.person', 'Work.trips.person',
       'Non.work.trips.person', 'New.COVID.cases', 'Population',
       'X..change.in.consumption', 'Transit.mode.share',
       'X..people.older.than.60', 'Median.income', 'X..African.Americans',
       'X..Hispanic.Americans', 'X..Male', 'Employment.density',
       'X..hot.spots.1000.people', 'Hospital.beds.1000.peop

In [ ]:
CA_labeled_data['growth_code'] = CA_labeled_data['growth_label'].astype('category').cat.codes

CA_labeled_data.head(2)

,Date,Province_State,FIPS,New cases,County Name,mask_rule_severity,m50,m50_index,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,PopDensity,State_Name,State,CTFIPS,CTNAME,STFIPS,Social.distancing.index,X..staying.home,Trips.person,X..out.of.county.trips,X..out.of.state.trips,Miles.person,Work.trips.person,Non.work.trips.person,New.COVID.cases,Population,X..change.in.consumption,Transit.mode.share,X..people.older.than.60,Median.income,X..African.Americans,X..Hispanic.Americans,X..Male,Employment.density,X..hot.spots.1000.people,Hospital.beds.1000.people,ICUs.1000.people,X..contact.tracing.workers.1000.people,New.cases.1000.people,Active.cases.1000.people,X.days..decreasing.COVID.cases,X..hospital.bed.utilization,Testing.capacity,Tests.done.1000.people,X..ICU.utilization,Ventilator.shortage,Imported.COVID.cases,COVID.exposure.1000.people,X.days..decreasing.ILI.cases,Unemployment.claims.1000.people,Unemployment.rate,X..working.from.home,Cumulative.inflation.rate,COVID.death.rate,New cases/100k,positivity_rate,rolling_avg_new_cases/100k,growth_label,growth_code
0,2020-03-01,California,6001,0,Alameda County,0.0,3.684,58,6.0,9.0,15.0,1.0,4.0,0.0,860.757075,California,CA,6001,Alameda County,6,37,26,2.93,25.4,0.4,36.8,0.25,2.68,0,1666753,-5.1,15.04,19,92574,10.5,22.5,49.11,1155,131,2.05,0.22,0.0,0.0,0.0,37,54.0,0.0,0.0,0.0,0,0,0.05,35,1.1,5.3,5.7,0.6,0.0,0.0,0.0,0.0,minimal,0
1,2020-03-02,California,6001,0,Alameda County,0.0,6.182,98,4.0,7.0,7.0,-1.0,3.0,0.0,860.757075,California,CA,6001,Alameda County,6,17,20,3.38,28.2,0.4,38.1,0.63,2.74,0,1666753,-2.9,15.04,19,92574,10.5,22.5,49.11,1155,131,2.05,0.22,0.0,0.0,0.0,38,54.0,0.0,0.0,0.0,0,0,0.05,35,1.1,5.3,5.7,0.6,0.0,0.0,0.0,0.0,minimal,0


In [ ]:
x_col_demographics = ['PopDensity','X..people.older.than.60',
                      'Median.income', 'X..African.Americans',
                      'X..Hispanic.Americans', 'X..Male','New cases','New.cases.1000.people']

feature_labels = ['FIPS','PopDensity','X..people.older.than.60',
                      'Median.income', 'X..African.Americans',
                      'X..Hispanic.Americans', 'X..Male']

target_column = ['growth_code']

X = CA_labeled_data[feature_labels]
y = CA_labeled_data[target_column]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)


In [ ]:
# Create a random forest classifier
clf = RandomForestClassifier(n_estimators=10000, random_state=0, n_jobs=-1)

# Train the classifier
clf.fit(X_train, y_train)

# Print the name and gini importance of each feature
for feature in zip(feature_labels, clf.feature_importances_):
    print(feature)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


('FIPS', 0.0874080002727691)
('PopDensity', 0.09777770512162384)
('X..people.older.than.60', 0.19726229104211354)
('Median.income', 0.09978030517826054)
('X..African.Americans', 0.08631257795640143)
('X..Hispanic.Americans', 0.34971446784318383)
('X..Male', 0.08174465258564782)


In [ ]:

#  Supported criteria are “gini” for the Gini impurity and “entropy” for the information gain. 
clf = RandomForestClassifier(n_estimators=10000, random_state=0,criterion='gini', n_jobs=-1)

# Train the classifier
clf.fit(X_train, y_train)

# Print the name and gini importance of each feature
for feature in zip(feature_labels, clf.feature_importances_):
    print(feature)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


('FIPS', 0.0874080002727691)
('PopDensity', 0.09777770512162384)
('X..people.older.than.60', 0.19726229104211354)
('Median.income', 0.09978030517826054)
('X..African.Americans', 0.08631257795640143)
('X..Hispanic.Americans', 0.34971446784318383)
('X..Male', 0.08174465258564782)


## Feature Importance: Extra tree classifier

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import RFE

In [ ]:
x_col_demographics = ['PopDensity','X..people.older.than.60',
                      'Median.income', 'X..African.Americans',
                      'X..Hispanic.Americans', 'X..Male','New cases','New.cases.1000.people']

feature_labels = ['FIPS','PopDensity','X..people.older.than.60',
                      'Median.income', 'X..African.Americans',
                      'X..Hispanic.Americans', 'X..Male']

target_column = ['growth_code']

X = CA_labeled_data[feature_labels]
y = CA_labeled_data[target_column]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

In [ ]:
# Create a random forest classifier
clf = RandomForestClassifier(n_estimators=10000, random_state=0, n_jobs=-1)

# Train the classifier
clf.fit(X_train, y_train)

# Print the name and gini importance of each feature
for feature in zip(feature_labels, clf.feature_importances_):
    print(feature)


#  Supported criteria are “gini” for the Gini impurity and “entropy” for the information gain. 
clf = RandomForestClassifier(n_estimators=10000, random_state=0,criterion='gini', n_jobs=-1)# Train the classifier
clf.fit(X_train, y_train)
# Print the name and gini importance of each feature
for feature in zip(feature_labels, clf.feature_importances_):
    print(feature)


model = ExtraTreesClassifier()
model.fit(X, y.values.ravel())
# display the relative importance of each attribute
#print(model.feature_importances_)


# create the RFE model 
rfe = RFE(model, 3)
rfe = rfe.fit(X, y.values.ravel())
# summarize the selection of the attributes
#print(rfe.support_)

print("\nFeatures with ranking:\n")
for index in range(len(rfe.ranking_)):
  print(feature_labels[index], rfe.ranking_[index])

# for index in range(len(rfe.ranking_)):
#   print( rfe.ranking_[index])
#print(rfe.ranking_)
#print(rfe.n_features_)
#rfe.n_features_to_select

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


('FIPS', 0.0874080002727691)
('PopDensity', 0.09777770512162384)
('X..people.older.than.60', 0.19726229104211354)
('Median.income', 0.09978030517826054)
('X..African.Americans', 0.08631257795640143)
('X..Hispanic.Americans', 0.34971446784318383)
('X..Male', 0.08174465258564782)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


('FIPS', 0.0874080002727691)
('PopDensity', 0.09777770512162384)
('X..people.older.than.60', 0.19726229104211354)
('Median.income', 0.09978030517826054)
('X..African.Americans', 0.08631257795640143)
('X..Hispanic.Americans', 0.34971446784318383)
('X..Male', 0.08174465258564782)

Features with ranking:

FIPS 4
PopDensity 5
X..people.older.than.60 1
Median.income 1
X..African.Americans 3
X..Hispanic.Americans 1
X..Male 2


#Hospitalization and testing Group

## Feature Importance: Gini-Coefficient

In [ ]:
CA_superset_data.columns

Index(['Date', 'Province_State', 'FIPS', 'New cases', 'County Name',
       'mask_rule_severity', 'm50', 'm50_index',
       'retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline',
       'parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline',
       'workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline', 'PopDensity', 'State_Name',
       'State', 'CTFIPS', 'CTNAME', 'STFIPS', 'Social.distancing.index',
       'X..staying.home', 'Trips.person', 'X..out.of.county.trips',
       'X..out.of.state.trips', 'Miles.person', 'Work.trips.person',
       'Non.work.trips.person', 'New.COVID.cases', 'Population',
       'X..change.in.consumption', 'Transit.mode.share',
       'X..people.older.than.60', 'Median.income', 'X..African.Americans',
       'X..Hispanic.Americans', 'X..Male', 'Employment.density',
       'X..hot.spots.1000.people', 'Hospital.beds.1000.peop

In [ ]:
def gini(list_of_values):
    sorted_list = sorted(list_of_values)
    #print (sorted_list)
    height, area = 0, 0
    for value in sorted_list:
        
        height += value
        area += height - value / 2.
    
    fair_area = height * len(list_of_values) / 2.
    print(fair_area)
    return (fair_area - area) / fair_area

In [ ]:
x_col_hospitalization_testing = ['Hospital.beds.1000.people',
       'ICUs.1000.people', 
       'X..hospital.bed.utilization',
       'Testing.capacity', 'Tests.done.1000.people', 'X..ICU.utilization',
       'Ventilator.shortage','New cases','New.cases.1000.people'] #'X..contact.tracing.workers.1000.people',

In [ ]:
feature_cols= x_col_hospitalization_testing

gini_values = []
for col in feature_cols:
  #print(col)
  dataset = CA_superset_data[col].values.tolist()
  #print(dataset)
  value = gini(dataset)
  print('cols ',col,' - gini coeff is :',value)
  gini_values.append(value)

242804953.02494016
cols  Hospital.beds.1000.people  - gini coeff is : -8.628750104779691e-14
26057116.909992337
cols  ICUs.1000.people  - gini coeff is : -2.3818190088626247e-13
6956397389.468079
cols  X..hospital.bed.utilization  - gini coeff is : 0.03308998365992187
847893268.2000415
cols  Testing.capacity  - gini coeff is : 0.22260662795532127
27711428164.375595
cols  Tests.done.1000.people  - gini coeff is : 0.525695711903369
1850321257.0899777
cols  X..ICU.utilization  - gini coeff is : 0.2995730960210415
74971523352.5
cols  Ventilator.shortage  - gini coeff is : 0.35019564195802766
14339402229.5
cols  New cases  - gini coeff is : 0.8685321834670556
16852079.173250053
cols  New.cases.1000.people  - gini coeff is : 0.680904683056212


## Feature Importance: Random Forest classifier(Gini Score)

In [ ]:
CA_labeled_data.columns

Index(['Date', 'Province_State', 'FIPS', 'New cases', 'County Name',
       'mask_rule_severity', 'm50', 'm50_index',
       'retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline',
       'parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline',
       'workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline', 'PopDensity', 'State_Name',
       'State', 'CTFIPS', 'CTNAME', 'STFIPS', 'Social.distancing.index',
       'X..staying.home', 'Trips.person', 'X..out.of.county.trips',
       'X..out.of.state.trips', 'Miles.person', 'Work.trips.person',
       'Non.work.trips.person', 'New.COVID.cases', 'Population',
       'X..change.in.consumption', 'Transit.mode.share',
       'X..people.older.than.60', 'Median.income', 'X..African.Americans',
       'X..Hispanic.Americans', 'X..Male', 'Employment.density',
       'X..hot.spots.1000.people', 'Hospital.beds.1000.peop

In [ ]:
CA_labeled_data['growth_code'] = CA_labeled_data['growth_label'].astype('category').cat.codes

CA_labeled_data.head(2)

,Date,Province_State,FIPS,New cases,County Name,mask_rule_severity,m50,m50_index,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,PopDensity,State_Name,State,CTFIPS,CTNAME,STFIPS,Social.distancing.index,X..staying.home,Trips.person,X..out.of.county.trips,X..out.of.state.trips,Miles.person,Work.trips.person,Non.work.trips.person,New.COVID.cases,Population,X..change.in.consumption,Transit.mode.share,X..people.older.than.60,Median.income,X..African.Americans,X..Hispanic.Americans,X..Male,Employment.density,X..hot.spots.1000.people,Hospital.beds.1000.people,ICUs.1000.people,X..contact.tracing.workers.1000.people,New.cases.1000.people,Active.cases.1000.people,X.days..decreasing.COVID.cases,X..hospital.bed.utilization,Testing.capacity,Tests.done.1000.people,X..ICU.utilization,Ventilator.shortage,Imported.COVID.cases,COVID.exposure.1000.people,X.days..decreasing.ILI.cases,Unemployment.claims.1000.people,Unemployment.rate,X..working.from.home,Cumulative.inflation.rate,COVID.death.rate,New cases/100k,positivity_rate,rolling_avg_new_cases/100k,growth_label,growth_code
0,2020-03-01,California,6001,0,Alameda County,0.0,3.684,58,6.0,9.0,15.0,1.0,4.0,0.0,860.757075,California,CA,6001,Alameda County,6,37,26,2.93,25.4,0.4,36.8,0.25,2.68,0,1666753,-5.1,15.04,19,92574,10.5,22.5,49.11,1155,131,2.05,0.22,0.0,0.0,0.0,37,54.0,0.0,0.0,0.0,0,0,0.05,35,1.1,5.3,5.7,0.6,0.0,0.0,0.0,0.0,minimal,0
1,2020-03-02,California,6001,0,Alameda County,0.0,6.182,98,4.0,7.0,7.0,-1.0,3.0,0.0,860.757075,California,CA,6001,Alameda County,6,17,20,3.38,28.2,0.4,38.1,0.63,2.74,0,1666753,-2.9,15.04,19,92574,10.5,22.5,49.11,1155,131,2.05,0.22,0.0,0.0,0.0,38,54.0,0.0,0.0,0.0,0,0,0.05,35,1.1,5.3,5.7,0.6,0.0,0.0,0.0,0.0,minimal,0


In [ ]:
x_col_hospitalization_testing = ['Hospital.beds.1000.people',
       'ICUs.1000.people', 
       'X..hospital.bed.utilization',
       'Testing.capacity', 'Tests.done.1000.people', 'X..ICU.utilization',
       'Ventilator.shortage','New cases','New.cases.1000.people'] #'X..contact.tracing.workers.1000.people',

feature_labels = ['FIPS','Hospital.beds.1000.people',
       'ICUs.1000.people', 
       'X..hospital.bed.utilization',
       'Testing.capacity', 'Tests.done.1000.people', 'X..ICU.utilization',
       'Ventilator.shortage']

target_column = ['growth_code']

X = CA_labeled_data[feature_labels]
y = CA_labeled_data[target_column]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)


In [ ]:
# Create a random forest classifier
clf = RandomForestClassifier(n_estimators=10000, random_state=0, n_jobs=-1)

# Train the classifier
clf.fit(X_train, y_train)

# Print the name and gini importance of each feature
for feature in zip(feature_labels, clf.feature_importances_):
    print(feature)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


('FIPS', 0.5564545734240648)
('Hospital.beds.1000.people', 0.0)
('ICUs.1000.people', 0.0)
('X..hospital.bed.utilization', 0.11437962824685657)
('Testing.capacity', 0.04374082430257774)
('Tests.done.1000.people', 0.13165942383332738)
('X..ICU.utilization', 0.09272099431546503)
('Ventilator.shortage', 0.06104455587770836)


In [ ]:

#  Supported criteria are “gini” for the Gini impurity and “entropy” for the information gain. 
clf = RandomForestClassifier(n_estimators=10000, random_state=0,criterion='gini', n_jobs=-1)

# Train the classifier
clf.fit(X_train, y_train)

# Print the name and gini importance of each feature
for feature in zip(feature_labels, clf.feature_importances_):
    print(feature)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


('FIPS', 0.5564545734240648)
('Hospital.beds.1000.people', 0.0)
('ICUs.1000.people', 0.0)
('X..hospital.bed.utilization', 0.11437962824685657)
('Testing.capacity', 0.04374082430257774)
('Tests.done.1000.people', 0.13165942383332738)
('X..ICU.utilization', 0.09272099431546503)
('Ventilator.shortage', 0.06104455587770836)


## Feature Importance: Extra tree classifier

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import RFE

In [ ]:
x_col_hospitalization_testing = ['Hospital.beds.1000.people',
       'ICUs.1000.people', 
       'X..hospital.bed.utilization',
       'Testing.capacity', 'Tests.done.1000.people', 'X..ICU.utilization',
       'Ventilator.shortage','New cases','New.cases.1000.people'] #'X..contact.tracing.workers.1000.people',

feature_labels = ['FIPS','Hospital.beds.1000.people',
       'ICUs.1000.people', 
       'X..hospital.bed.utilization',
       'Testing.capacity', 'Tests.done.1000.people', 'X..ICU.utilization',
       'Ventilator.shortage']

target_column = ['growth_code']

X = CA_labeled_data[feature_labels]
y = CA_labeled_data[target_column]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

In [ ]:
# Create a random forest classifier
clf = RandomForestClassifier(n_estimators=10000, random_state=0, n_jobs=-1)

# Train the classifier
clf.fit(X_train, y_train)

# Print the name and gini importance of each feature
for feature in zip(feature_labels, clf.feature_importances_):
    print(feature)


#  Supported criteria are “gini” for the Gini impurity and “entropy” for the information gain. 
clf = RandomForestClassifier(n_estimators=10000, random_state=0,criterion='gini', n_jobs=-1)# Train the classifier
clf.fit(X_train, y_train)
# Print the name and gini importance of each feature
for feature in zip(feature_labels, clf.feature_importances_):
    print(feature)


model = ExtraTreesClassifier()
model.fit(X, y.values.ravel())
# display the relative importance of each attribute
#print(model.feature_importances_)


# create the RFE model 
rfe = RFE(model, 3)
rfe = rfe.fit(X, y.values.ravel())
# summarize the selection of the attributes
#print(rfe.support_)

print("\nFeatures with ranking:\n")
for index in range(len(rfe.ranking_)):
  print(feature_labels[index], rfe.ranking_[index])

# for index in range(len(rfe.ranking_)):
#   print( rfe.ranking_[index])
#print(rfe.ranking_)
#print(rfe.n_features_)
#rfe.n_features_to_select

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


('FIPS', 0.5564545734240648)
('Hospital.beds.1000.people', 0.0)
('ICUs.1000.people', 0.0)
('X..hospital.bed.utilization', 0.11437962824685657)
('Testing.capacity', 0.04374082430257774)
('Tests.done.1000.people', 0.13165942383332738)
('X..ICU.utilization', 0.09272099431546503)
('Ventilator.shortage', 0.06104455587770836)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


('FIPS', 0.5564545734240648)
('Hospital.beds.1000.people', 0.0)
('ICUs.1000.people', 0.0)
('X..hospital.bed.utilization', 0.11437962824685657)
('Testing.capacity', 0.04374082430257774)
('Tests.done.1000.people', 0.13165942383332738)
('X..ICU.utilization', 0.09272099431546503)
('Ventilator.shortage', 0.06104455587770836)

Features with ranking:

FIPS 1
Hospital.beds.1000.people 6
ICUs.1000.people 5
X..hospital.bed.utilization 1
Testing.capacity 4
Tests.done.1000.people 1
X..ICU.utilization 2
Ventilator.shortage 3


# Mobility Group 

## Feature Importance: Gini-Coefficient

In [ ]:
CA_superset_data.columns

Index(['Date', 'Province_State', 'FIPS', 'New cases', 'County Name',
       'mask_rule_severity', 'm50', 'm50_index',
       'retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline',
       'parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline',
       'workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline', 'PopDensity', 'State_Name',
       'State', 'CTFIPS', 'CTNAME', 'STFIPS', 'Social.distancing.index',
       'X..staying.home', 'Trips.person', 'X..out.of.county.trips',
       'X..out.of.state.trips', 'Miles.person', 'Work.trips.person',
       'Non.work.trips.person', 'New.COVID.cases', 'Population',
       'X..change.in.consumption', 'Transit.mode.share',
       'X..people.older.than.60', 'Median.income', 'X..African.Americans',
       'X..Hispanic.Americans', 'X..Male', 'Employment.density',
       'X..hot.spots.1000.people', 'Hospital.beds.1000.peop

In [ ]:
def gini(list_of_values):
    sorted_list = sorted(list_of_values)
    #print (sorted_list)
    height, area = 0, 0
    for value in sorted_list:
        
        height += value
        area += height - value / 2.
    
    fair_area = height * len(list_of_values) / 2.
    print(fair_area)
    return (fair_area - area) / fair_area

In [ ]:
x_col_mobility = ['retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline','parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline','workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline',
       'X..staying.home', 'Trips.person', 'X..out.of.county.trips',
       'X..out.of.state.trips', 'Miles.person', 'Work.trips.person',
       'Non.work.trips.person','X..working.from.home','New cases','New.cases.1000.people']

In [ ]:
feature_cols= x_col_mobility

gini_values = []
for col in feature_cols:
  #print(col)
  dataset = CA_superset_data[col].values.tolist()
  #print(dataset)
  value = gini(dataset)
  print('cols ',col,' - gini coeff is :',value)
  gini_values.append(value)

nan
cols  retail_and_recreation_percent_change_from_baseline  - gini coeff is : nan
nan
cols  grocery_and_pharmacy_percent_change_from_baseline  - gini coeff is : nan
nan
cols  parks_percent_change_from_baseline  - gini coeff is : nan
nan
cols  transit_stations_percent_change_from_baseline  - gini coeff is : nan
nan
cols  workplaces_percent_change_from_baseline  - gini coeff is : nan
nan
cols  residential_percent_change_from_baseline  - gini coeff is : nan
3185659962.0
cols  X..staying.home  - gini coeff is : 0.1173911693215423
357092208.7150005
cols  Trips.person  - gini coeff is : 0.07230888148726836
2776322465.0999947
cols  X..out.of.county.trips  - gini coeff is : 0.24785794461926924
204607954.00000012
cols  X..out.of.state.trips  - gini coeff is : 0.7522604458475726
4698066581.149977
cols  Miles.person  - gini coeff is : 0.18458958295302125
44415963.439996496
cols  Work.trips.person  - gini coeff is : 0.1805772735479392
312679323.47500366
cols  Non.work.trips.person  - gini coeff 

## Feature Importance: Random Forest classifier(Gini Score)

In [ ]:
CA_labeled_data.columns

Index(['Date', 'Province_State', 'FIPS', 'New cases', 'County Name',
       'mask_rule_severity', 'm50', 'm50_index',
       'retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline',
       'parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline',
       'workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline', 'PopDensity', 'State_Name',
       'State', 'CTFIPS', 'CTNAME', 'STFIPS', 'Social.distancing.index',
       'X..staying.home', 'Trips.person', 'X..out.of.county.trips',
       'X..out.of.state.trips', 'Miles.person', 'Work.trips.person',
       'Non.work.trips.person', 'New.COVID.cases', 'Population',
       'X..change.in.consumption', 'Transit.mode.share',
       'X..people.older.than.60', 'Median.income', 'X..African.Americans',
       'X..Hispanic.Americans', 'X..Male', 'Employment.density',
       'X..hot.spots.1000.people', 'Hospital.beds.1000.peop

In [ ]:
CA_labeled_data['growth_code'] = CA_labeled_data['growth_label'].astype('category').cat.codes

CA_labeled_data.head(2)

,Date,Province_State,FIPS,New cases,County Name,mask_rule_severity,m50,m50_index,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,PopDensity,State_Name,State,CTFIPS,CTNAME,STFIPS,Social.distancing.index,X..staying.home,Trips.person,X..out.of.county.trips,X..out.of.state.trips,Miles.person,Work.trips.person,Non.work.trips.person,New.COVID.cases,Population,X..change.in.consumption,Transit.mode.share,X..people.older.than.60,Median.income,X..African.Americans,X..Hispanic.Americans,X..Male,Employment.density,X..hot.spots.1000.people,Hospital.beds.1000.people,ICUs.1000.people,X..contact.tracing.workers.1000.people,New.cases.1000.people,Active.cases.1000.people,X.days..decreasing.COVID.cases,X..hospital.bed.utilization,Testing.capacity,Tests.done.1000.people,X..ICU.utilization,Ventilator.shortage,Imported.COVID.cases,COVID.exposure.1000.people,X.days..decreasing.ILI.cases,Unemployment.claims.1000.people,Unemployment.rate,X..working.from.home,Cumulative.inflation.rate,COVID.death.rate,New cases/100k,positivity_rate,rolling_avg_new_cases/100k,growth_label,growth_code
0,2020-03-01,California,6001,0,Alameda County,0.0,3.684,58,6.0,9.0,15.0,1.0,4.0,0.0,860.757075,California,CA,6001,Alameda County,6,37,26,2.93,25.4,0.4,36.8,0.25,2.68,0,1666753,-5.1,15.04,19,92574,10.5,22.5,49.11,1155,131,2.05,0.22,0.0,0.0,0.0,37,54.0,0.0,0.0,0.0,0,0,0.05,35,1.1,5.3,5.7,0.6,0.0,0.0,0.0,0.0,minimal,0
1,2020-03-02,California,6001,0,Alameda County,0.0,6.182,98,4.0,7.0,7.0,-1.0,3.0,0.0,860.757075,California,CA,6001,Alameda County,6,17,20,3.38,28.2,0.4,38.1,0.63,2.74,0,1666753,-2.9,15.04,19,92574,10.5,22.5,49.11,1155,131,2.05,0.22,0.0,0.0,0.0,38,54.0,0.0,0.0,0.0,0,0,0.05,35,1.1,5.3,5.7,0.6,0.0,0.0,0.0,0.0,minimal,0


In [ ]:
x_col_mobility = ['retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline','parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline','workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline',
       'X..staying.home', 'Trips.person', 'X..out.of.county.trips',
       'X..out.of.state.trips', 'Miles.person', 'Work.trips.person',
       'Non.work.trips.person','X..working.from.home','New cases','New.cases.1000.people']

feature_labels = ['FIPS','retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline','parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline','workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline',
       'X..staying.home', 'Trips.person', 'X..out.of.county.trips',
       'X..out.of.state.trips', 'Miles.person', 'Work.trips.person',
       'Non.work.trips.person','X..working.from.home']

target_column = ['growth_code']

X = CA_labeled_data[feature_labels]
y = CA_labeled_data[target_column]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)


In [ ]:
# Create a random forest classifier
clf = RandomForestClassifier(n_estimators=10000, random_state=0, n_jobs=-1)

# Train the classifier
clf.fit(X_train, y_train)

# Print the name and gini importance of each feature
for feature in zip(feature_labels, clf.feature_importances_):
    print(feature)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


('FIPS', 0.05017484916482331)
('retail_and_recreation_percent_change_from_baseline', 0.07234835764825823)
('grocery_and_pharmacy_percent_change_from_baseline', 0.0654454699185266)
('parks_percent_change_from_baseline', 0.07412945852462671)
('transit_stations_percent_change_from_baseline', 0.06350954617078809)
('workplaces_percent_change_from_baseline', 0.059527588756943736)
('residential_percent_change_from_baseline', 0.047259678500213086)
('X..staying.home', 0.04261355860939692)
('Trips.person', 0.0508649018718169)
('X..out.of.county.trips', 0.06458724363074085)
('X..out.of.state.trips', 0.04865995031054884)
('Miles.person', 0.08490202275333326)
('Work.trips.person', 0.0528962943046745)
('Non.work.trips.person', 0.04947413031391449)
('X..working.from.home', 0.17360694952139463)


In [ ]:

#  Supported criteria are “gini” for the Gini impurity and “entropy” for the information gain. 
clf = RandomForestClassifier(n_estimators=10000, random_state=0,criterion='gini', n_jobs=-1)

# Train the classifier
clf.fit(X_train, y_train)

# Print the name and gini importance of each feature
for feature in zip(feature_labels, clf.feature_importances_):
    print(feature)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


('FIPS', 0.05017484916482331)
('retail_and_recreation_percent_change_from_baseline', 0.07234835764825823)
('grocery_and_pharmacy_percent_change_from_baseline', 0.0654454699185266)
('parks_percent_change_from_baseline', 0.07412945852462671)
('transit_stations_percent_change_from_baseline', 0.06350954617078809)
('workplaces_percent_change_from_baseline', 0.059527588756943736)
('residential_percent_change_from_baseline', 0.047259678500213086)
('X..staying.home', 0.04261355860939692)
('Trips.person', 0.0508649018718169)
('X..out.of.county.trips', 0.06458724363074085)
('X..out.of.state.trips', 0.04865995031054884)
('Miles.person', 0.08490202275333326)
('Work.trips.person', 0.0528962943046745)
('Non.work.trips.person', 0.04947413031391449)
('X..working.from.home', 0.17360694952139463)


## Feature Importance: Extra tree classifier

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import RFE

In [ ]:
x_col_mobility = ['retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline','parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline','workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline',
       'X..staying.home', 'Trips.person', 'X..out.of.county.trips',
       'X..out.of.state.trips', 'Miles.person', 'Work.trips.person',
       'Non.work.trips.person','X..working.from.home','New cases','New.cases.1000.people']

feature_labels = ['FIPS','retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline','parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline','workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline',
       'X..staying.home', 'Trips.person', 'X..out.of.county.trips',
       'X..out.of.state.trips', 'Miles.person', 'Work.trips.person',
       'Non.work.trips.person','X..working.from.home']

target_column = ['growth_code']

X = CA_labeled_data[feature_labels]
y = CA_labeled_data[target_column]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

In [ ]:
# Create a random forest classifier
clf = RandomForestClassifier(n_estimators=10000, random_state=0, n_jobs=-1)

# Train the classifier
clf.fit(X_train, y_train)

# Print the name and gini importance of each feature
for feature in zip(feature_labels, clf.feature_importances_):
    print(feature)


#  Supported criteria are “gini” for the Gini impurity and “entropy” for the information gain. 
clf = RandomForestClassifier(n_estimators=10000, random_state=0,criterion='gini', n_jobs=-1)# Train the classifier
clf.fit(X_train, y_train)
# Print the name and gini importance of each feature
for feature in zip(feature_labels, clf.feature_importances_):
    print(feature)


model = ExtraTreesClassifier()
model.fit(X, y.values.ravel())
# display the relative importance of each attribute
#print(model.feature_importances_)


# create the RFE model 
rfe = RFE(model, 3)
rfe = rfe.fit(X, y.values.ravel())
# summarize the selection of the attributes
#print(rfe.support_)

print("\nFeatures with ranking:\n")
for index in range(len(rfe.ranking_)):
  print(feature_labels[index], rfe.ranking_[index])

# for index in range(len(rfe.ranking_)):
#   print( rfe.ranking_[index])
#print(rfe.ranking_)
#print(rfe.n_features_)
#rfe.n_features_to_select

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


('FIPS', 0.05017484916482331)
('retail_and_recreation_percent_change_from_baseline', 0.07234835764825823)
('grocery_and_pharmacy_percent_change_from_baseline', 0.0654454699185266)
('parks_percent_change_from_baseline', 0.07412945852462671)
('transit_stations_percent_change_from_baseline', 0.06350954617078809)
('workplaces_percent_change_from_baseline', 0.059527588756943736)
('residential_percent_change_from_baseline', 0.047259678500213086)
('X..staying.home', 0.04261355860939692)
('Trips.person', 0.0508649018718169)
('X..out.of.county.trips', 0.06458724363074085)
('X..out.of.state.trips', 0.04865995031054884)
('Miles.person', 0.08490202275333326)
('Work.trips.person', 0.0528962943046745)
('Non.work.trips.person', 0.04947413031391449)
('X..working.from.home', 0.17360694952139463)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


('FIPS', 0.05017484916482331)
('retail_and_recreation_percent_change_from_baseline', 0.07234835764825823)
('grocery_and_pharmacy_percent_change_from_baseline', 0.0654454699185266)
('parks_percent_change_from_baseline', 0.07412945852462671)
('transit_stations_percent_change_from_baseline', 0.06350954617078809)
('workplaces_percent_change_from_baseline', 0.059527588756943736)
('residential_percent_change_from_baseline', 0.047259678500213086)
('X..staying.home', 0.04261355860939692)
('Trips.person', 0.0508649018718169)
('X..out.of.county.trips', 0.06458724363074085)
('X..out.of.state.trips', 0.04865995031054884)
('Miles.person', 0.08490202275333326)
('Work.trips.person', 0.0528962943046745)
('Non.work.trips.person', 0.04947413031391449)
('X..working.from.home', 0.17360694952139463)

Features with ranking:

FIPS 6
retail_and_recreation_percent_change_from_baseline 2
grocery_and_pharmacy_percent_change_from_baseline 7
parks_percent_change_from_baseline 3
transit_stations_percent_change_from

# All Groups Together 

## Feature Importance: Gini-Coefficient

In [37]:
CA_superset_data.columns

Index(['Date', 'Province_State', 'FIPS', 'New cases', 'County Name',
       'mask_rule_severity', 'm50', 'm50_index',
       'retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline',
       'parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline',
       'workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline', 'PopDensity', 'State_Name',
       'State', 'CTFIPS', 'CTNAME', 'STFIPS', 'Social.distancing.index',
       'X..staying.home', 'Trips.person', 'X..out.of.county.trips',
       'X..out.of.state.trips', 'Miles.person', 'Work.trips.person',
       'Non.work.trips.person', 'New.COVID.cases', 'Population',
       'X..change.in.consumption', 'Transit.mode.share',
       'X..people.older.than.60', 'Median.income', 'X..African.Americans',
       'X..Hispanic.Americans', 'X..Male', 'Employment.density',
       'X..hot.spots.1000.people', 'Hospital.beds.1000.peop

In [38]:
def gini(list_of_values):
    sorted_list = sorted(list_of_values)
    #print (sorted_list)
    height, area = 0, 0
    for value in sorted_list:
        
        height += value
        area += height - value / 2.
    
    fair_area = height * len(list_of_values) / 2.
    print(fair_area)
    return (fair_area - area) / fair_area

In [39]:
x_col_all = ['FIPS','mask_rule_severity','m50_index','retail_and_recreation_percent_change_from_baseline',
                'grocery_and_pharmacy_percent_change_from_baseline','parks_percent_change_from_baseline',
                'transit_stations_percent_change_from_baseline','workplaces_percent_change_from_baseline',
                'residential_percent_change_from_baseline','PopDensity','Social.distancing.index',
                'X..staying.home','Trips.person','X..out.of.county.trips',	'X..out.of.state.trips',
                'Miles.person',	'Work.trips.person','Non.work.trips.person','X..change.in.consumption',
                'Transit.mode.share','X..people.older.than.60',	'Median.income',	'X..African.Americans',
                'X..Hispanic.Americans',	'X..Male','Employment.density',	'X..hot.spots.1000.people',
                'Hospital.beds.1000.people',	'ICUs.1000.people',
                'New.cases.1000.people',	'Active.cases.1000.people',	'X.days..decreasing.COVID.cases',
                'X..hospital.bed.utilization',	'Testing.capacity',	'Tests.done.1000.people',	'X..ICU.utilization',
                'Ventilator.shortage',	'Imported.COVID.cases',	'COVID.exposure.1000.people',	'X.days..decreasing.ILI.cases',
                'Unemployment.claims.1000.people',	'Unemployment.rate',	'X..working.from.home',
                'Cumulative.inflation.rate'	,'COVID.death.rate']

In [40]:
feature_cols= x_col_all

gini_values = []
for col in feature_cols:
  #print(col)
  dataset = CA_superset_data[col].values.tolist()
  #print(dataset)
  value = gini(dataset)
  print('cols ',col,' - gini coeff is :',value)
  gini_values.append(value)

717532829521.5
cols  FIPS  - gini coeff is : 0.0031471682814930156
745540040.0
cols  mask_rule_severity  - gini coeff is : 0.37054122539146256
5624456258.0
cols  m50_index  - gini coeff is : 0.41370009104264954
nan
cols  retail_and_recreation_percent_change_from_baseline  - gini coeff is : nan
nan
cols  grocery_and_pharmacy_percent_change_from_baseline  - gini coeff is : nan
nan
cols  parks_percent_change_from_baseline  - gini coeff is : nan
nan
cols  transit_stations_percent_change_from_baseline  - gini coeff is : nan
nan
cols  workplaces_percent_change_from_baseline  - gini coeff is : nan
nan
cols  residential_percent_change_from_baseline  - gini coeff is : nan
35575489453.94788
cols  PopDensity  - gini coeff is : 0.8156637311902494
4389836411.0
cols  Social.distancing.index  - gini coeff is : 0.18524526015646098
3185659962.0
cols  X..staying.home  - gini coeff is : 0.1173911693215423
357092208.7150005
cols  Trips.person  - gini coeff is : 0.07230888148726836
2776322465.0999947
cols 

## Feature Importance: Random Forest classifier(Gini Score)

In [41]:
CA_labeled_data.columns

Index(['Date', 'Province_State', 'FIPS', 'New cases', 'County Name',
       'mask_rule_severity', 'm50', 'm50_index',
       'retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline',
       'parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline',
       'workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline', 'PopDensity', 'State_Name',
       'State', 'CTFIPS', 'CTNAME', 'STFIPS', 'Social.distancing.index',
       'X..staying.home', 'Trips.person', 'X..out.of.county.trips',
       'X..out.of.state.trips', 'Miles.person', 'Work.trips.person',
       'Non.work.trips.person', 'New.COVID.cases', 'Population',
       'X..change.in.consumption', 'Transit.mode.share',
       'X..people.older.than.60', 'Median.income', 'X..African.Americans',
       'X..Hispanic.Americans', 'X..Male', 'Employment.density',
       'X..hot.spots.1000.people', 'Hospital.beds.1000.peop

In [42]:
CA_labeled_data['growth_code'] = CA_labeled_data['growth_label'].astype('category').cat.codes

CA_labeled_data.head(2)

,Date,Province_State,FIPS,New cases,County Name,mask_rule_severity,m50,m50_index,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,PopDensity,State_Name,State,CTFIPS,CTNAME,STFIPS,Social.distancing.index,X..staying.home,Trips.person,X..out.of.county.trips,X..out.of.state.trips,Miles.person,Work.trips.person,Non.work.trips.person,New.COVID.cases,Population,X..change.in.consumption,Transit.mode.share,X..people.older.than.60,Median.income,X..African.Americans,X..Hispanic.Americans,X..Male,Employment.density,X..hot.spots.1000.people,Hospital.beds.1000.people,ICUs.1000.people,X..contact.tracing.workers.1000.people,New.cases.1000.people,Active.cases.1000.people,X.days..decreasing.COVID.cases,X..hospital.bed.utilization,Testing.capacity,Tests.done.1000.people,X..ICU.utilization,Ventilator.shortage,Imported.COVID.cases,COVID.exposure.1000.people,X.days..decreasing.ILI.cases,Unemployment.claims.1000.people,Unemployment.rate,X..working.from.home,Cumulative.inflation.rate,COVID.death.rate,New cases/100k,positivity_rate,rolling_avg_new_cases/100k,growth_label,growth_code
0,2020-03-01,California,6001,0,Alameda County,0.0,3.684,58,6.0,9.0,15.0,1.0,4.0,0.0,860.757075,California,CA,6001,Alameda County,6,37,26,2.93,25.4,0.4,36.8,0.25,2.68,0,1666753,-5.1,15.04,19,92574,10.5,22.5,49.11,1155,131,2.05,0.22,0.0,0.0,0.0,37,54.0,0.0,0.0,0.0,0,0,0.05,35,1.1,5.3,5.7,0.6,0.0,0.0,0.0,0.0,minimal,0
1,2020-03-02,California,6001,0,Alameda County,0.0,6.182,98,4.0,7.0,7.0,-1.0,3.0,0.0,860.757075,California,CA,6001,Alameda County,6,17,20,3.38,28.2,0.4,38.1,0.63,2.74,0,1666753,-2.9,15.04,19,92574,10.5,22.5,49.11,1155,131,2.05,0.22,0.0,0.0,0.0,38,54.0,0.0,0.0,0.0,0,0,0.05,35,1.1,5.3,5.7,0.6,0.0,0.0,0.0,0.0,minimal,0


In [43]:


feature_labels = ['FIPS','mask_rule_severity','m50_index','retail_and_recreation_percent_change_from_baseline',
                'grocery_and_pharmacy_percent_change_from_baseline','parks_percent_change_from_baseline',
                'transit_stations_percent_change_from_baseline','workplaces_percent_change_from_baseline',
                'residential_percent_change_from_baseline','PopDensity','Social.distancing.index',
                'X..staying.home','Trips.person','X..out.of.county.trips',	'X..out.of.state.trips',
                'Miles.person',	'Work.trips.person','Non.work.trips.person','X..change.in.consumption',
                'Transit.mode.share','X..people.older.than.60',	'Median.income',	'X..African.Americans',
                'X..Hispanic.Americans',	'X..Male','Employment.density',	'X..hot.spots.1000.people',
                'Hospital.beds.1000.people',	'ICUs.1000.people',
                'New.cases.1000.people',	'Active.cases.1000.people',	'X.days..decreasing.COVID.cases',
                'X..hospital.bed.utilization',	'Testing.capacity',	'Tests.done.1000.people',	'X..ICU.utilization',
                'Ventilator.shortage',	'Imported.COVID.cases',	'COVID.exposure.1000.people',	'X.days..decreasing.ILI.cases',
                'Unemployment.claims.1000.people',	'Unemployment.rate',	'X..working.from.home',
                'Cumulative.inflation.rate'	,'COVID.death.rate',
                	'positivity_rate',	'rolling_avg_new_cases/100k']

target_column = ['growth_code']

X = CA_labeled_data[feature_labels]
y = CA_labeled_data[target_column]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)


In [44]:
# Create a random forest classifier
clf = RandomForestClassifier(n_estimators=10000, random_state=0, n_jobs=-1)

# Train the classifier
clf.fit(X_train, y_train)

# Print the name and gini importance of each feature
for feature in zip(feature_labels, clf.feature_importances_):
    print(feature)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


('FIPS', 0.005427347573881217)
('mask_rule_severity', 0.010561423140285518)
('m50_index', 0.004734975606412593)
('retail_and_recreation_percent_change_from_baseline', 0.005224398121747533)
('grocery_and_pharmacy_percent_change_from_baseline', 0.004665346032139882)
('parks_percent_change_from_baseline', 0.005783859325271695)
('transit_stations_percent_change_from_baseline', 0.005341784155548222)
('workplaces_percent_change_from_baseline', 0.003477763768351752)
('residential_percent_change_from_baseline', 0.0031140841784784317)
('PopDensity', 0.0054781904163756145)
('Social.distancing.index', 0.0032658647335110716)
('X..staying.home', 0.0029279706167285036)
('Trips.person', 0.0036150876202827114)
('X..out.of.county.trips', 0.0054658067538416516)
('X..out.of.state.trips', 0.0046348299386258195)
('Miles.person', 0.005854713526640946)
('Work.trips.person', 0.00330061306156646)
('Non.work.trips.person', 0.0037340176425873324)
('X..change.in.consumption', 0.0036449930184031996)
('Transit.mode

In [45]:

#  Supported criteria are “gini” for the Gini impurity and “entropy” for the information gain. 
clf = RandomForestClassifier(n_estimators=10000, random_state=0,criterion='gini', n_jobs=-1)

# Train the classifier
clf.fit(X_train, y_train)

# Print the name and gini importance of each feature
for feature in zip(feature_labels, clf.feature_importances_):
    print(feature)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


('FIPS', 0.005427347573881217)
('mask_rule_severity', 0.010561423140285518)
('m50_index', 0.004734975606412593)
('retail_and_recreation_percent_change_from_baseline', 0.005224398121747533)
('grocery_and_pharmacy_percent_change_from_baseline', 0.004665346032139882)
('parks_percent_change_from_baseline', 0.005783859325271695)
('transit_stations_percent_change_from_baseline', 0.005341784155548222)
('workplaces_percent_change_from_baseline', 0.003477763768351752)
('residential_percent_change_from_baseline', 0.0031140841784784317)
('PopDensity', 0.0054781904163756145)
('Social.distancing.index', 0.0032658647335110716)
('X..staying.home', 0.0029279706167285036)
('Trips.person', 0.0036150876202827114)
('X..out.of.county.trips', 0.0054658067538416516)
('X..out.of.state.trips', 0.0046348299386258195)
('Miles.person', 0.005854713526640946)
('Work.trips.person', 0.00330061306156646)
('Non.work.trips.person', 0.0037340176425873324)
('X..change.in.consumption', 0.0036449930184031996)
('Transit.mode

## Feature Importance: Extra tree classifier

In [46]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import RFE

In [47]:


feature_labels = ['FIPS','mask_rule_severity','m50_index','retail_and_recreation_percent_change_from_baseline',
                'grocery_and_pharmacy_percent_change_from_baseline','parks_percent_change_from_baseline',
                'transit_stations_percent_change_from_baseline','workplaces_percent_change_from_baseline',
                'residential_percent_change_from_baseline','PopDensity','Social.distancing.index',
                'X..staying.home','Trips.person','X..out.of.county.trips',	'X..out.of.state.trips',
                'Miles.person',	'Work.trips.person','Non.work.trips.person','X..change.in.consumption',
                'Transit.mode.share','X..people.older.than.60',	'Median.income',	'X..African.Americans',
                'X..Hispanic.Americans',	'X..Male','Employment.density',	'X..hot.spots.1000.people',
                'Hospital.beds.1000.people',	'ICUs.1000.people',
                'New.cases.1000.people',	'Active.cases.1000.people',	'X.days..decreasing.COVID.cases',
                'X..hospital.bed.utilization',	'Testing.capacity',	'Tests.done.1000.people',	'X..ICU.utilization',
                'Ventilator.shortage',	'Imported.COVID.cases',	'COVID.exposure.1000.people',	'X.days..decreasing.ILI.cases',
                'Unemployment.claims.1000.people',	'Unemployment.rate',	'X..working.from.home',
                'Cumulative.inflation.rate'	,'COVID.death.rate',
                	'positivity_rate',	'rolling_avg_new_cases/100k']

target_column = ['growth_code']

X = CA_labeled_data[feature_labels]
y = CA_labeled_data[target_column]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

In [48]:
# Create a random forest classifier
clf = RandomForestClassifier(n_estimators=10000, random_state=0, n_jobs=-1)

# Train the classifier
clf.fit(X_train, y_train)

# Print the name and gini importance of each feature
for feature in zip(feature_labels, clf.feature_importances_):
    print(feature)


#  Supported criteria are “gini” for the Gini impurity and “entropy” for the information gain. 
clf = RandomForestClassifier(n_estimators=10000, random_state=0,criterion='gini', n_jobs=-1)# Train the classifier
clf.fit(X_train, y_train)
# Print the name and gini importance of each feature
for feature in zip(feature_labels, clf.feature_importances_):
    print(feature)


model = ExtraTreesClassifier()
model.fit(X, y.values.ravel())
# display the relative importance of each attribute
#print(model.feature_importances_)


# create the RFE model 
rfe = RFE(model, 3)
rfe = rfe.fit(X, y.values.ravel())
# summarize the selection of the attributes
#print(rfe.support_)

print("\nFeatures with ranking:\n")
for index in range(len(rfe.ranking_)):
  print(feature_labels[index], rfe.ranking_[index])

# for index in range(len(rfe.ranking_)):
#   print( rfe.ranking_[index])
#print(rfe.ranking_)
#print(rfe.n_features_)
#rfe.n_features_to_select

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


('FIPS', 0.005427347573881217)
('mask_rule_severity', 0.010561423140285518)
('m50_index', 0.004734975606412593)
('retail_and_recreation_percent_change_from_baseline', 0.005224398121747533)
('grocery_and_pharmacy_percent_change_from_baseline', 0.004665346032139882)
('parks_percent_change_from_baseline', 0.005783859325271695)
('transit_stations_percent_change_from_baseline', 0.005341784155548222)
('workplaces_percent_change_from_baseline', 0.003477763768351752)
('residential_percent_change_from_baseline', 0.0031140841784784317)
('PopDensity', 0.0054781904163756145)
('Social.distancing.index', 0.0032658647335110716)
('X..staying.home', 0.0029279706167285036)
('Trips.person', 0.0036150876202827114)
('X..out.of.county.trips', 0.0054658067538416516)
('X..out.of.state.trips', 0.0046348299386258195)
('Miles.person', 0.005854713526640946)
('Work.trips.person', 0.00330061306156646)
('Non.work.trips.person', 0.0037340176425873324)
('X..change.in.consumption', 0.0036449930184031996)
('Transit.mode

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


('FIPS', 0.005427347573881217)
('mask_rule_severity', 0.010561423140285518)
('m50_index', 0.004734975606412593)
('retail_and_recreation_percent_change_from_baseline', 0.005224398121747533)
('grocery_and_pharmacy_percent_change_from_baseline', 0.004665346032139882)
('parks_percent_change_from_baseline', 0.005783859325271695)
('transit_stations_percent_change_from_baseline', 0.005341784155548222)
('workplaces_percent_change_from_baseline', 0.003477763768351752)
('residential_percent_change_from_baseline', 0.0031140841784784317)
('PopDensity', 0.0054781904163756145)
('Social.distancing.index', 0.0032658647335110716)
('X..staying.home', 0.0029279706167285036)
('Trips.person', 0.0036150876202827114)
('X..out.of.county.trips', 0.0054658067538416516)
('X..out.of.state.trips', 0.0046348299386258195)
('Miles.person', 0.005854713526640946)
('Work.trips.person', 0.00330061306156646)
('Non.work.trips.person', 0.0037340176425873324)
('X..change.in.consumption', 0.0036449930184031996)
('Transit.mode

# Conclusion for all groups superset dataset analysis:

From the above Gini approches, we can drop the following columns from Time-series forecasting


## Gini-Coefficient


>* FIPS  - gini coeff is : 0.0031471682814930156
>* mask_rule_severity  - gini coeff is : 0.37054122539146256
>* m50_index  - gini coeff is : 0.41370009104264954
>* retail_and_recreation_percent_change_from_baseline  - gini coeff is : nan
nan
>* grocery_and_pharmacy_percent_change_from_baseline  - gini coeff is : nan
nan
>* parks_percent_change_from_baseline  - gini coeff is : nan
nan
>* transit_stations_percent_change_from_baseline  - gini coeff is : nan
nan
>* workplaces_percent_change_from_baseline  - gini coeff is : nan
nan
>* residential_percent_change_from_baseline  - gini coeff is : nan
>* PopDensity  - gini coeff is : 0.8156637311902494
>* Social.distancing.index  - gini coeff is : 0.18524526015646098
>* X..staying.home  - gini coeff is : 0.1173911693215423
>* Trips.person  - gini coeff is : 0.07230888148726836
>*  X..out.of.county.trips  - gini coeff is : 0.24785794461926924
>* X..out.of.state.trips  - gini coeff is : 0.7522604458475726
>* Miles.person  - gini coeff is : 0.18458958295302125
>* Work.trips.person  - gini coeff is : 0.1805772735479392
>* Non.work.trips.person  - gini coeff is : 0.07466161452747519
>* X..change.in.consumption  - gini coeff is : -0.939130643149599
>* Transit.mode.share  - gini coeff is : 0.6464346063619306
>* X..people.older.than.60  - gini coeff is : 0.14673423679461214
>* Median.income  - gini coeff is : 0.15433249671121319
>* X..African.Americans  - gini coeff is : 0.46161607629775014
>* X..Hispanic.Americans  - gini coeff is : 0.3070973510639532
>* X..Male  - gini coeff is : 0.017576291485370876
>* Employment.density  - gini coeff is : 0.7695110721560425
>* X..hot.spots.1000.people  - gini coeff is : 0.16486568548969588
>* Hospital.beds.1000.people  - gini coeff is : -8.628750104779691e-14
>* ICUs.1000.people  - gini coeff is : -2.3818190088626247e-13
>* New.cases.1000.people  - gini coeff is : 0.680904683056212
>* Active.cases.1000.people  - gini coeff is : 0.5046967465588664
>* X.days..decreasing.COVID.cases  - gini coeff is : 0.833358411533779
>* X..hospital.bed.utilization  - gini coeff is : 0.03308998365992187
>* Testing.capacity  - gini coeff is : 0.22260662795532127
>* Tests.done.1000.people  - gini coeff is : 0.525695711903369
>*  X..ICU.utilization  - gini coeff is : 0.2995730960210415
>* Ventilator.shortage  - gini coeff is : 0.35019564195802766
>* Imported.COVID.cases  - gini coeff is : 0.7502043415588123
>* COVID.exposure.1000.people  - gini coeff is : 0.5789358250538936
>* X.days..decreasing.ILI.cases  - gini coeff is : 0.3914747299774321
>*  Unemployment.claims.1000.people  - gini coeff is : 0.29723080034346144
>* Unemployment.rate  - gini coeff is : 0.15105700504555333
>* X..working.from.home  - gini coeff is : 0.08548800988546724
>* Cumulative.inflation.rate  - gini coeff is : 0.31801944797755666
>*  COVID.death.rate  - gini coeff is : 0.4712221652206614

## Random Forest

>* ('FIPS', 0.005427347573881217)
>* ('mask_rule_severity', 0.010561423140285518)
>* ('m50_index', 0.004734975606412593)
>* ('retail_and_recreation_percent_change_from_baseline', 0.005224398121747533)
>* ('grocery_and_pharmacy_percent_change_from_baseline', 0.004665346032139882)
>* ('parks_percent_change_from_baseline', 0.005783859325271695)
>* ('transit_stations_percent_change_from_baseline', 0.005341784155548222)
>* ('workplaces_percent_change_from_baseline', 0.003477763768351752)
>* ('residential_percent_change_from_baseline', 0.0031140841784784317)
>* ('PopDensity', 0.0054781904163756145)
>* ('Social.distancing.index', 0.0032658647335110716)
>* ('X..staying.home', 0.0029279706167285036)
>* ('Trips.person', 0.0036150876202827114)
>* ('X..out.of.county.trips', 0.0054658067538416516)
>* ('X..out.of.state.trips', 0.0046348299386258195)
>* ('Miles.person', 0.005854713526640946)
>* ('Work.trips.person', 0.00330061306156646)
>* ('Non.work.trips.person', 0.0037340176425873324)
>* ('X..change.in.consumption', 0.0036449930184031996)
>* ('Transit.mode.share', 0.0065957692778026485)
>* ('X..people.older.than.60', 0.01043936811732236)
>* ('Median.income', 0.007984587653261105)
>* ('X..African.Americans', 0.005235972568456647)
>* ('X..Hispanic.Americans', 0.019035274704033347)
>* ('X..Male', 0.0069089116282343085)
>* ('Employment.density', 0.0054538536004907255)
>* ('X..hot.spots.1000.people', 0.014787795725153478)
>* ('Hospital.beds.1000.people', 0.0)
>* ('ICUs.1000.people', 0.0)
>* ('New.cases.1000.people', 0.14803656961971212)
>* ('Active.cases.1000.people', 0.04532160465912705)
>* ('X.days..decreasing.COVID.cases', 0.0030003295303095505)
>* ('X..hospital.bed.utilization', 0.028331151109518754)
>* ('Testing.capacity', 0.006259111288755067)
>* ('Tests.done.1000.people', 0.03446430941900461)
>* ('X..ICU.utilization', 0.02303164713879015)
>* ('Ventilator.shortage', 0.014250902443406648)
>* ('Imported.COVID.cases', 0.013621871309362099)
>* ('COVID.exposure.1000.people', 0.0346008009120861)
>* ('X.days..decreasing.ILI.cases', 0.014310261085923629)
>* ('Unemployment.claims.1000.people', 0.008034399458611891)
>* ('Unemployment.rate', 0.006251946721400913)
>* ('X..working.from.home', 0.009029153055485434)
>* ('Cumulative.inflation.rate', 0.015507510501708578)
>* ('COVID.death.rate', 0.015464850050174181)
>* ('positivity_rate', 0.046870152440507226)
>* ('rolling_avg_new_cases/100k', 0.3669188267506812)

## Extra Tree Classifier- with rank

>* rolling_avg_new_cases/100k 1
>* New.cases.1000.people 1
>* Active.cases.1000.people 1
>* Tests.done.1000.people 2
>* X..Hispanic.Americans 3
>* COVID.exposure.1000.people 4
>* positivity_rate 5
>* mask_rule_severity 6
>* X..hot.spots.1000.people 7
>* X..ICU.utilization 8
>* X..hospital.bed.utilization 9
>* X.days..decreasing.ILI.cases 10
>* X..people.older.than.60 11
>* Median.income 12
>* Ventilator.shortage 13
>* Imported.COVID.cases 14
>* COVID.death.rate 15
>* Cumulative.inflation.rate 16
>* Transit.mode.share 17
>* X..Male 18
>* FIPS 19
>* X..African.Americans 20
>* Unemployment.rate 21
>* X..working.from.home 22
>* Employment.density 23
>* transit_stations_percent_change_from_baseline 24
>* PopDensity 25
>* Unemployment.claims.1000.people 26
>* X..out.of.state.trips 27
>* Testing.capacity 28
>* X..out.of.county.trips 29
>* parks_percent_change_from_baseline 30
>* retail_and_recreation_percent_change_from_baseline 31
>* grocery_and_pharmacy_percent_change_from_baseline 32
>* X.days..decreasing.COVID.cases 33
>* Miles.person 34
>* residential_percent_change_from_baseline 35
>* workplaces_percent_change_from_baseline 36
>* X..staying.home 37
>* Social.distancing.index 38
>* m50_index 39
>* Trips.person 40
>* Work.trips.person 41
>* Non.work.trips.person 42
>* X..change.in.consumption 43
>* ICUs.1000.people 44
>* Hospital.beds.1000.people 45





















